In [1]:
# import dataset
import pandas as pd
import numpy as np

data_latih = pd.read_csv("train.csv")

In [2]:
# mengetahui jumlah missing value dari setiap kolom
for col in data_latih.columns.tolist():
    print('{} column missing values: {}'.format(col, data_latih[col].isnull().sum()))

job_level column missing values: 0
job_duration_in_current_job_level column missing values: 0
person_level column missing values: 0
job_duration_in_current_person_level column missing values: 0
job_duration_in_current_branch column missing values: 0
Employee_type column missing values: 0
gender column missing values: 0
age column missing values: 0
marital_status_maried(Y/N) column missing values: 0
number_of_dependences column missing values: 0
Education_level column missing values: 0
GPA column missing values: 0
year_graduated column missing values: 0
job_duration_from_training column missing values: 0
branch_rotation column missing values: 0
job_rotation column missing values: 0
assign_of_otherposition column missing values: 0
annual leave column missing values: 0
sick_leaves column missing values: 0
Last_achievement_% column missing values: 1
Achievement_above_100%_during3quartal column missing values: 1
Best Performance column missing values: 0


In [3]:
# mengecek apakah terdapat data kosong (Null/NaN)
print(data_latih.isnull().values.any())

# menghapus missing value
data_latih = data_latih.dropna(axis=0, how="any")

# mengecek apakah terdapat data kosong (Null/NaN)
print(data_latih.isnull().values.any())

True
False


In [4]:
# melakukan encoding untuk data kategorikal
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
kategorik = ['job_level','person_level','Education_level','Employee_type','marital_status_maried(Y/N)']

for fitur in kategorik:
    data_latih[fitur] = LabelEncoder().fit_transform(data_latih[fitur])

In [6]:
# pemodelan
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = StandardScaler().fit_transform(data_latih.drop("Best Performance", axis = 1))
y = data_latih["Best Performance"]
X_latih, X_uji, y_latih, y_uji = train_test_split(X, y, test_size=0.3)
model = RandomForestClassifier().fit(X_latih, y_latih)

y_pred_latih = model.predict(X_latih)
akurasi = accuracy_score(y_latih, y_pred_latih)
print('akurasi latih:', akurasi)

y_pred_uji = model.predict(X_uji)
akurasi = accuracy_score(y_uji, y_pred_uji)
print('akurasi latih:', akurasi)

akurasi latih: 1.0
akurasi latih: 0.8556485355648535


In [7]:
print(y_pred_uji)

[0 0 0 ... 0 0 0]


In [ ]:
## feature selection ##2
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

test = SelectKBest(score_func=f_classif)
fit = test.fit(X_latih, y_latih)
# summarize scores
set_printoptions(precision=3)
rank2 = zip(fit.scores_, data_latih.drop("Best Performance", axis = 1).columns)
sorted(rank2, key=lambda t: t[0], reverse=True)

fitur yang memiliki rank terendah dari dua metode feature selection:
- number_of_dependences
- job_rotation
- assign_of_otherposition
- Achievement_above_100%_during3quartal
- person_level
- job_level

In [ ]:
# drop fitur yang tida diperlukan
fitur = ["number_of_dependences","job_rotation","assign_of_otherposition",
         "Achievement_above_100%_during3quartal","person_level","job_level"]
semua_data = list(map(lambda d:d.drop(fitur, axis = 1), [data_latih, data_uji]))

# standardscaler dan pemisahan fitur target
standard_scaler = StandardScaler()
standard_scaler.fit(semua_data[0].drop("Best Performance", axis = 1))
X_latih = standard_scaler.transform(semua_data[0].drop("Best Performance", axis = 1))
y_latih = semua_data[0]["Best Performance"]
X_test = standard_scaler.transform(semua_data[1])

# training
model = RandomForestRegressor().fit(X_latih, y_latih)

y_pred_latih = model.predict(X_latih)
AUC = roc_auc_score(y_latih, y_pred_latih)
print(AUC)

In [ ]:
# prediksi
y_pred_uji = model.predict(X_test)
submission = pd.read_csv("submission.csv")
submission["Best Performance"] = y_pred_uji
submission

In [ ]:
submission.to_csv("submission_v3.csv", index=False)